In [1]:
!pip install -r ./requirements.txt -q

In [2]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.function_step import step
from sagemaker import get_execution_role, Session
import logging
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
from steps.fetch import fetch
from steps.preprocess import preprocess
from steps.train import train
from steps.eval import eval
from steps.deploy import deploy

In [12]:
# # mport importlib
# # import sys
# # importlib.reload(sys.modules['my_module'])
# # from my_module import my_function

# import importlib
# import steps.deploy
# importlib.reload(steps.deploy)
# from steps.deploy import deploy

In [4]:
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

In [5]:
sagemaker_session = Session()
role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()
session = PipelineSession(boto_session=sagemaker_session.boto_session, default_bucket=default_bucket)
pipeline_name = 'stock-pipeline'
default_ticker = "SPY"
years_of_data_to_fetch = 1

sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/aai-540_stock_trading_ai/cloud


In [6]:
fetch_result = fetch(default_ticker, years_of_data_to_fetch)
# print(fetch_result)

In [9]:
preprocess_result = preprocess(fetch_result)
# print(preprocess_result)

In [10]:
train_result = train(preprocess_result)
# print(train_result)

In [11]:
eval_result = eval(train_result)
# print(eval_result)

In [12]:
deploy_result = deploy(eval_result)
# print(deploy_result)

In [13]:
pipeline = Pipeline(
    name=pipeline_name,
    steps=[fetch_result, preprocess_result, train_result, eval_result, deploy_result],
    sagemaker_session=session,
)

In [14]:
pipeline.upsert(role)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-02-22 19:49:57,431 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-381e2240-ea15-4f74-87a5-59955f831116/2025-02-22-19-49-55-409/function
2025-02-22 19:49:57,536 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-381e2240-ea15-4f74-87a5-59955f831116/2025-02-22-19-49-55-409/arguments
2025-02-22 19:49:57,728 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmph_u485ec/requirements.txt'
2025-02-22 19:49:57,758 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-381e2240-ea15-4f74-87a5-59955f831116/2025-02-22-19-49-55-409/pre_exec_script_and_dependencies'
2025-02-22 19:49:57,764 sagemaker.remote_function INFO     Copied user workspace to '/tmp/tmpjgy26d8z/temp_workspace/

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-02-22 19:49:59,679 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/preprocess-06555a88-57b5-45b4-9f1d-80016ac4edc3/2025-02-22-19-49-55-409/function
2025-02-22 19:49:59,758 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/preprocess-06555a88-57b5-45b4-9f1d-80016ac4edc3/2025-02-22-19-49-55-409/arguments
2025-02-22 19:49:59,833 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmp66hdy4br/requirements.txt'
2025-02-22 19:49:59,861 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-971173012767/stock-pipeline/preprocess-06555a88-57b5-45b4-9f1d-80016ac4edc3/2025-02-22-19-49-55-409/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


...

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-02-22 19:50:05,256 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/eval-6412e598-ac0a-4b06-ac99-29abc411c1f4/2025-02-22-19-49-55-409/function
2025-02-22 19:50:05,322 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/eval-6412e598-ac0a-4b06-ac99-29abc411c1f4/2025-02-22-19-49-55-409/arguments
2025-02-22 19:50:05,400 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmp6q3uewe_/requirements.txt'
2025-02-22 19:50:05,428 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-971173012767/stock-pipeline/eval-6412e598-ac0a-4b06-ac99-29abc411c1f4/2025-02-22-19-49-55-409/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-02-22 19:50:07,700 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/deploy-e9cee4d3-93ea-4862-9893-721e0f29d111/2025-02-22-19-49-55-409/function
2025-02-22 19:50:07,765 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/deploy-e9cee4d3-93ea-4862-9893-721e0f29d111/2025-02-22-19-49-55-409/arguments
2025-02-22 19:50:07,834 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmp85pbdlnq/requirements.txt'
2025-02-22 19:50:07,864 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-971173012767/stock-pipeline/deploy-e9cee4d3-93ea-4862-9893-721e0f29d111/2025-02-22-19-49-55-409/pre_exec_script_and_dependencies'
2025-02-22 19:50:08,531 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:971173012767:pipeline/stock-pipeline',
 'ResponseMetadata': {'RequestId': '1e1ce4d3-1d9a-40f2-8b49-11654fb8b8b4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1e1ce4d3-1d9a-40f2-8b49-11654fb8b8b4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Sat, 22 Feb 2025 19:50:09 GMT'},
  'RetryAttempts': 0}}

In [15]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:971173012767:pipeline/stock-pipeline/execution/42i3hk5gvopy', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x7f13701fa8d0>)

## Verify after successful pipeline run

In [ ]:
from steps.deploy import deploy
import boto3
import json
import torch
import numpy as np
from datetime import datetime
import logging

In [ ]:
# Set to endpoint name
endpoint_name="stock-pipeline-endpoint-20250221-234058"

In [ ]:
sequence_length = 30
input_data = np.random.rand(sequence_length, 4).tolist()  
# Ensure the input matches the expected shape: [seq_len, 4]
# endpoint_name="https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/stock-pipeline-endpoint-20250221-234058/invocatio
endpoint_name="stock-pipeline-endpoint-20250221-234058"

# Convert to JSON
input_json = json.dumps(input_data)

# Call the endpoint
try:
    runtime = boto3.client('sagemaker-runtime')
    print(f"Invoking endpoint: {endpoint_name}")
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=input_json
    )
    
    # Parse the response
    result = json.loads(response['Body'].read().decode())
    # logger.info(f"Prediction: {result}")
    print(f"Prediction from endpoint {endpoint_name}: {result}")

except Exception as e:
    print(f"Error invoking endpoint: {e}")
    raise e